In [14]:
%config IPCompleter.greedy = True # for code auto-completion

In [15]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import time
from tqdm import tqdm
from displayfunction import display


In [16]:
geolocator = Nominatim(timeout=10, user_agent = "onos.pkp") # ! an absolute maximum of 1 request per second

In [17]:
# example
location = geolocator.geocode('Wrocław Główny')
display(location)
print(location.latitude)
print(location.longitude)
display(location.raw)

Wrocław Główny, tunel boczny, Glinianki, Osiedle Huby, Wrocław, województwo dolnośląskie, 50-000, Polska
(51.097916, 17.0379512)
51.097916
17.0379512
place_id
licence
osm_type
osm_id
boundingbox
lat
lon
display_name
class
type
importance
icon


In [18]:
ic_nodes = pd.read_csv('ic.nodes')
ic_nodes

,id,name
0,0,ALEKSANDRÓW KUJAWSKI
1,1,ANDRYCHÓW
2,2,AUGUSTÓW
3,3,BABIAK
4,4,BABIMOST
...,...,...
454,454,ŻMIGRÓD
455,455,ŻORY
456,456,ŻYCHLIN
457,457,ŻYRARDÓW


In [19]:
get_coordinates_from_api = False # change to True, if you want coordinates update

if get_coordinates_from_api:
    
    data_with_coord = pd.DataFrame(columns=['id','name','latitude','longitude'])

    time_per_request = 1.2

    data = ic_nodes
    # data = ic_nodes.head(4) # for testing
#     data = ic_nodes[150:170]
    
    empty = []
    class Location(object):
        latitude = 0
        longitude = 0
    
    with tqdm(total=len(data.index)) as pbar:
        for index, row in data.iterrows():

            time.sleep(time_per_request)
            location = geolocator.geocode(row['name']+', Polska')  # ! an absolute maximum of 1 request per second
            
            if location is None:
                empty.append(row['name'])
                location = Location()
                
            data_with_coord = data_with_coord.append(
                {'id': row['id'],
                 'name': row['name'],
                 'latitude': location.latitude,
                 'longitude': location.longitude
                }
                , ignore_index=True)
            pbar.update(1)
    
    display(empty) # handle manualy
    # None's: 'MAŁASZEWICZE PRZYSTANEK', 'WARSZAWA ZACHODNIA PERON 8'
    
    data_with_coord.to_csv('ic_coord.nodes', index=False)
    display(data_with_coord)

In [20]:
ic_nodes_coord = pd.read_csv('ic_coord.nodes')

manual_fix = [
    ('MAŁASZEWICZE PRZYSTANEK', 52.027912, 23.5297023),
    ('WARSZAWA ZACHODNIA PERON 8', 52.221139, 20.9590913),
    ('KOSTRZYN', 52.591157, 14.6473684),
    ('KRZYŻ', 52.8344443,16.0049416),
    
]

# weird coordinates 

for name, lat, lon in manual_fix:
    ic_nodes_coord.loc[ic_nodes_coord['name'] == name, 'latitude'] = lat
    ic_nodes_coord.loc[ic_nodes_coord['name'] == name, 'longitude'] = lon

ic_nodes_coord.to_csv('ic_coord.nodes', index=False)

display(ic_nodes_coord[165:170])
display(ic_nodes_coord)

id
name
latitude
longitude
id
name
latitude
longitude
